In [ ]:
# Install & imports
!pip install openai jsonschema --quiet

import os
import time
import json
from typing import List, Dict, Any, Optional
import openai
import requests
from jsonschema import validate, ValidationError


In [ ]:
from openai import OpenAI
import os

# set Groq API key
os.environ["GROQ_API_KEY"] = "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"

client = OpenAI(
    api_key=os.environ["GROQ_API_KEY"],
    base_url="https://api.groq.com/openai/v1"
)

In [ ]:
# Api Test
resp = client.chat.completions.create(
    model="llama-3.1-8b-instant",
    messages=[{"role": "user", "content": "explain how Groq work simply"}]
)
print(resp.choices[0].message.content)


Groq is a company specializing in chip and software solutions for accelerating machine learning (ML) workloads. I'll break down how they work simply:

**Hardware Component: Groq Chip**

Groq's AI chip is designed to improve the performance and efficiency of ML computations. The chip consists of multiple key components:

1. **Highly Parallel Architecture**: It has thousands of processors and AI cores that can run many operations simultaneously, accelerating training and inference tasks.
2. **High Bandwidth Memory**: The chip has access to high-speed memory that reduces data transfer bottlenecks, allowing for faster access to data.
3. **Low Latency and High Throughput**: The chip is optimized for low latency and high throughput, making it suitable for real-time AI applications.

**Software Component: Groq Platform**

The Groq platform is a suite of software tools that work with the AI chip to accelerate ML workloads. It includes:

1. **Compiler and Runtime**: This part optimizes code for

In [ ]:
GROQ_API_KEY = os.environ.get("GROQ_API_KEY")

client = openai.OpenAI(
    api_key=GROQ_API_KEY,
    base_url="https://api.groq.com/openai/v1"
)

GROQ_BASE = "https://api.groq.com/openai/v1"
headers = {
    "Authorization": f"Bearer {GROQ_API_KEY}",
    "Content-Type": "application/json",
}


In [ ]:
from openai import OpenAI
import os

# configure Groq client once
client = OpenAI(
    api_key=os.environ["GROQ_API_KEY"],
    base_url="https://api.groq.com/openai/v1"
)

class ConversationManager:
    def __init__(self, model="llama-3.1-8b-instant", summarization_trigger_k=3):
        self.model = model
        self.history = []
        self.summarization_trigger_k = summarization_trigger_k
        self.append_count = 0

    def append(self, role, content):
        self.history.append({"role": role, "content": content})
        self.append_count += 1

        # check if summarization should trigger
        if self.append_count % self.summarization_trigger_k == 0:
            summary = self.summarize()
            self._replace_summary(summary)
            return {"summarized": True, "summary": summary}
        else:
            return {"summarized": False}

    def summarize(self):
        try:
            response = client.chat.completions.create(
                model=self.model,
                messages=[
                    {"role": "system", "content": "Summarize this conversation concisely."},
                    {"role": "user", "content": str(self.history)}
                ],
            )
            # Correct extraction
            return response.choices[0].message.content
        except Exception as e:
            return f"(fallback summary due to API error: {e})"

    def _replace_summary(self, summary):
        # remove any existing auto-summary
        self.history = [msg for msg in self.history if not (
            msg["role"] == "system" and msg["content"].startswith("Conversation summary (auto):")
        )]
        # insert new one at the start
        self.history.insert(0, {
            "role": "system",
            "content": f"Conversation summary (auto):\n{summary}"
        })

    def truncate_last_n_turns(self, n):
        # keep system + last n turns
        system_msgs = [m for m in self.history if m["role"] == "system"]
        non_system_msgs = [m for m in self.history if m["role"] != "system"]
        self.history = system_msgs + non_system_msgs[-n:]

    def get_history(self):
        return self.history

    def truncate_last_n_turns(self, n: int):
        # each "turn" here is considered one message. If you want one user+assistant as a turn, adjust accordingly.
        if n <= 0:
            return
        self.history = self.history[-n:]

    def truncate_by_char_limit(self, char_limit: int):
        # keep messages until char_limit is satisfied, prioritize recent messages
        new_hist = []
        total = 0
        for msg in reversed(self.history):
            length = len(msg["content"])
            if total + length > char_limit:
                break
            new_hist.append(msg)
            total += length
        self.history = list(reversed(new_hist))

    def truncate_by_word_limit(self, word_limit: int):
        new_hist = []
        total = 0
        for msg in reversed(self.history):
            words = len(msg["content"].split())
            if total + words > word_limit:
                break
            new_hist.append(msg)
            total += words
        self.history = list(reversed(new_hist))

    def summarize_history(self, prompt_prefix: Optional[str]=None, max_tokens:int=400) -> str:
        """
        Summarize current history by sending to the model.
        Returns plain text summary.
        """
        if not self.history:
            return ""

        # Prepare summary prompt: include the concatenated conversation with roles
        convo_text = "\n\n".join([f"{m['role'].upper()}: {m['content']}" for m in self.history])
        system_prompt = "You are a concise summarizer. Produce a short summary capturing the key points, actions, and any named entities."

        if prompt_prefix:
            user_prompt = prompt_prefix + "\n\nConversation:\n" + convo_text
        else:
            user_prompt = "Summarize the following conversation briefly, focusing on key facts, requested actions, and any personal information:\n\n" + convo_text

        # Use Groq via OpenAI-compatible client
        try:
            resp = client.chat.completions.create(
                model=self.summary_model,
                messages=[
                    {"role":"system","content":system_prompt},
                    {"role":"user","content":user_prompt}
                ],
                max_tokens=max_tokens,
                temperature=0.0,
            )
            # Groq returns choices similar to OpenAI; adapt if necessary
            # Attempt robust extraction:
            text = None
            if hasattr(resp, "choices"):
                text = resp.choices[0].message["content"]
            elif isinstance(resp, dict) and "choices" in resp:
                text = resp["choices"][0]["message"]["content"]
            elif hasattr(resp, "output") and resp.output:
                # some Groq endpoints return output
                text = resp.output[0].get("content", "")
            else:
                text = str(resp)
            return text.strip()
        except Exception as e:
            # Fallback to local simplistic summarizer (if no API key or call fails)
            # naive: take first and last message condensed
            fallback = " | ".join([self.history[0]["content"][:200], self.history[-1]["content"][:200]])
            return f"(fallback summary due to API error: {e}) {fallback}"


In [ ]:
cm = ConversationManager(model="llama-3.1-8b-instant", summarization_trigger_k=3)

# feed some conversation samples
samples = [
    ("user", "Hi, I want help with my project. It's an app to recommend fertilizers."),
    ("assistant", "Sure — what's the target platform and what data do you have?"),
    ("user", "Mobile, soil data and weather. I also want a summarizer module."),
    ("assistant", "Good. You should structure conversations and summarize periodically."),
    ("user", "Make an endpoint for summarization; keep logs."),
    ("assistant", "I'll draft the endpoint and schema."),
]

outputs = []
for r,c in samples:
    out = cm.append(r, c)
    outputs.append(out)

print("After feeding 5 messages (summarization runs at every 3rd append):")
print("Run outputs (shows if summarization happened):")
print(outputs)

print("\nCurrent history:")
for m in cm.get_history():
    print(m["role"], ":", m["content"][:200])

# Demonstrate truncation options
cm.append("user", "Extra message to trigger summarization (3rd run).")
print("\nAfter another append (should have triggered summarization at 6th run if k=3):")
for m in cm.get_history():
    print(m["role"], ":", (m["content"][:300] + '...') if len(m["content"])>300 else m["content"])

# Truncate example: keep last 2 messages
cm.truncate_last_n_turns(2)
print("\nAfter truncating to last 2 messages:")
for m in cm.get_history():
    print(m)


After feeding 5 messages (summarization runs at every 3rd append):
Run outputs (shows if summarization happened):
[{'summarized': False}, {'summarized': False}, {'summarized': True, 'summary': "You're developing an app that recommends fertilizers to users based on soil data and weather information, primarily for mobile platforms. You also want to include a summarizer module."}, {'summarized': False}, {'summarized': False}, {'summarized': True, 'summary': 'The conversation is about developing an app that recommends fertilizers based on soil data and weather information for mobile platforms. The key points include:\n\n- Developing an app for mobile platforms\n- Using soil data and weather information for fertilizer recommendations\n- Incorporating a summarizer module\n- Creating an endpoint for summarization and keeping logs'}]

Current history:
system : Conversation summary (auto):
The conversation is about developing an app that recommends fertilizers based on soil data and weather inf

In [ ]:
import json
from jsonschema import validate, ValidationError
from openai import OpenAI

# Groq client setup
client = OpenAI(
    api_key=os.environ.get("OPENAI_API_KEY"),
    base_url="https://api.groq.com/openai/v1"
)

# JSON Schema
person_schema = {
    "name": "person_info",
    "description": "Extract name, email, phone, location and age if present. Return null if missing.",
    "type": "object",
    "properties": {
        "name": {"type": ["string", "null"]},
        "email": {"type": ["string", "null"], "format": "email"},
        "phone": {"type": ["string", "null"]},
        "location": {"type": ["string", "null"]},
        "age": {"type": ["integer", "null"], "minimum": 0, "maximum": 130}
    },
    "required": []
}

functions = [
    {
        "name": "extract_person_info",
        "description": "Extract structured person info (name, email, phone, location, age) from a chat text.",
        "parameters": person_schema
    }
]

# Sample Chats
sample_chats = [
    "Hey, I'm Amit Deo. You can reach me at amit@example.com or on +91-9876543210. I'm 22 and based in Pune.",
    "Hello, this is Priya. My email is priya.work@mail.com. I live in Bengaluru. Age: 24.",
    "User: Hi. I'm interested, contact: 555-1234. No email yet. Name: Rahul, Location: Delhi."
]

# Function Caller
def call_function_extract(chat_text):
    """
    Call Groq API (OpenAI-compatible function calling) asking to 'call' extract_person_info
    """
    messages = [
        {"role": "system", "content": "You are a JSON extractor. Use the function with the exact schema to return valid JSON."},
        {"role": "user", "content": f"Extract info from this chat:\n\n{chat_text}"}
    ]
    try:
        resp = client.chat.completions.create(
            model="llama-3.1-8b-instant",
            messages=messages,
            functions=functions,
            function_call={"name": "extract_person_info"},  # force function call
            max_tokens=400,
            temperature=0.0,
        )

        choice = resp.choices[0].message

        if choice.function_call:  # model chose function call
            args_text = choice.function_call.arguments
            parsed = json.loads(args_text)
            return parsed, resp

        if choice.content:  # model may fallback to JSON content
            try:
                parsed = json.loads(choice.content)
                return parsed, resp
            except Exception:
                return {"error": "Could not parse JSON from model output", "raw": choice.content}, resp

        return {"error": "No usable output"}, resp

    except Exception as e:
        return {"error": str(e)}, None

# ---------------- Demo ----------------
for i, chat in enumerate(sample_chats, 1):
    parsed, raw_resp = call_function_extract(chat)
    print(f"\nSample {i} input:\n{chat}\nParsed output:\n{parsed}")
    try:
        validate(instance=parsed, schema=person_schema)
        print("Validation: OK")
    except ValidationError as ve:
        print("Validation ERROR:", ve)



Sample 1 input:
Hey, I'm Amit Deo. You can reach me at amit@example.com or on +91-9876543210. I'm 22 and based in Pune.
Parsed output:
{'age': 22, 'email': 'amit@example.com', 'location': 'Pune', 'name': 'Amit Deo', 'phone': '+91-9876543210'}
Validation: OK

Sample 2 input:
Hello, this is Priya. My email is priya.work@mail.com. I live in Bengaluru. Age: 24.
Parsed output:
{'age': 24, 'email': 'priya.work@mail.com', 'location': 'Bengaluru', 'name': 'Priya', 'phone': None}
Validation: OK

Sample 3 input:
User: Hi. I'm interested, contact: 555-1234. No email yet. Name: Rahul, Location: Delhi.
Parsed output:
{'age': None, 'email': None, 'location': 'Delhi', 'name': 'Rahul', 'phone': '555-1234'}
Validation: OK
